# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['AngleX', 'AngleY', 'LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [2]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [3]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Stand', 'Left', 'Crouch', 'Sit', 'Belly', 'Knee', 'StandInit', 'Back', 'Right', 'Frog', 'HeadBack']


In [4]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [41]:
# load all the data
all_data = []
all_target = []
count = 0

for i in range(0,len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    count += len(data)
    
    
print count
print 'total number of data', len(all_data)

200
total number of data 200


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [104]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.5)
training_data = permutation[:n_training_data]

In [105]:
print training_data

[ 44 106 191  21 116  97  15   7  85  30 172 164 165 179 178  48   4  93
 158   5 119 112  73 177  25 197  55  86 151  36 130 109 154  37  71 126
 160  79  91  54 144 127 103 167  53  33 173  28 113 195  78 194 161  26
  19 148  72 134 196  56 124  62 155  75 107 190 157 122  51  76 185 180
  18  43  80 139  64 186 129 149 132   3 184 100 188 183 131 199  94 192
 142  63 135  38   1 133 105 110 136 141]


In [106]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [107]:
fit_data = []
fit_target = []
for i in range(0,len(training_data)):
    fit_data.append(all_data[training_data[i]])
    fit_target.append(all_target[training_data[i]])
    
clf.fit(fit_data, fit_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [108]:
clf.predict(all_data[1]), all_target[1]

(array([0]), 0)

In [109]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [110]:
expected = []
predicted = []
prediction_data = []
for i in range(0,len(all_data)):
    if( not (i in training_data)):
        expected.append(all_target[i])
        prediction_data.append(all_data[i])
predicted = clf.predict(prediction_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         6
          1       1.00      1.00      1.00        12
          2       1.00      1.00      1.00        18
          3       1.00      1.00      1.00        13
          4       1.00      1.00      1.00         7
          5       1.00      1.00      1.00         5
          6       1.00      1.00      1.00        22
          7       1.00      1.00      1.00         5
          8       1.00      1.00      1.00         6
          9       1.00      1.00      1.00         4
         10       1.00      1.00      1.00         2

avg / total       1.00      1.00      1.00       100


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0]
 [ 0  0 18  0  0  0  0  0  0  0  0]
 [ 0  0  0 13  0  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  0 22  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [111]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(expected, predicted)

ZeroDivisionError: Weights sum to zero, can't be normalized

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [ ]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [ ]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]